In [ ]:
# !pip install tensorflow-datasets
!pip install -U tfds-nightly --user
# !pip install tensorflow-gpu

In [ ]:
# print(tf.__version__)
# !pip install tensorflow==2.* --user

In [ ]:
# !pip install -U --pre tensorflow=="2.1"
!pip install tf_slim

In [ ]:
!pip install pycocotools

In [ ]:
# !pip install tensorflow==2.1
# !pip install six

In [ ]:
import numpy as np
import os
import six
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
# import tensorflow.compat.v1 as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display



In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd models/research
pip install .

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
import tensorflow_datasets as tfds


In [ ]:
# ds = tfds.load('kitti', split='train',data_dir='/content/tensorflow_datasets', shuffle_files=True)
# assert isinstance(ds, tf.data.Dataset)
# print(ds)

In [ ]:
# !gcloud auth login application-default



In [ ]:
!gcloud config set account application-default

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# !gcloud auth login dissertation-0




In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'kitti',
    split=['train', 'test'],
#     shuffle_files=True,
    # as_supervised=True,
#     try_gcs=True,
    with_info=True,
    download=False,
#     data_dir='./tensorflow_datasets'
    data_dir="gs://kitti-dataset-1"
)

# ds = tfds.load('kitti', split='train',data_dir='/content/tensorflow_datasets')


In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [ ]:

# model_name = 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'
# model_name = 'ssd_inception_v2_coco_2018_01_28'
model_name = 'faster_rcnn_resnet50_lowproposals_coco_2018_01_28'
# model_name = 'faster_rcnn_resnet101_kitti_2018_01_28'
# model_name = 'faster_rcnn_inception_v2_coco_2018_01_28'
# model_name = 'faster_rcnn_nas_lowproposals_coco_2018_01_28'
# model_name = 'ssd_mobilenet_v1_coco_2018_01_28'
# model_name = 'faster_rcnn_resnet101_lowproposals_coco_2018_01_28'
detection_model = load_model(model_name)


In [ ]:
# List of the strings that is used to add correct label for each box.

# PATH_TO_LABELS = 'models/research/object_detection/data/kitti_label_map.pbtxt'
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


In [ ]:
category_index

In [ ]:
def normalize_img(object):
  print(object['objects'])
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(object['image'], tf.float32) / 255., object['objects']['bbox']



In [ ]:
# ds_train = ds_train.map(
#     normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
# ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
# ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
ds_train

In [ ]:
# ds_test = ds_test.map(
#     normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
print(detection_model.inputs)



In [ ]:
detection_model.output_dtypes

In [ ]:
detection_model.output_shapes

## Output format


- Output classes are always integers in the range 0, num_classes). Any mapping of these integers to semantic labels is to be handled outside of this class. We never explicitly emit a “background class” --- thus 0 is the first non-background class and any logic of predicting and removing implicit background classes must be handled internally by the implementation.


- Detected boxes are to be interpreted as being in (y_min, x_min, y_max, x_max) format and normalized relative to the image window.

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

 bbox: RawBoundingBox, bounding box in Kitti coordinates (origin top left).

In [ ]:
def norm(data):
    height= 375
    width = 1242
    
    data.loc[:,'xmin'] = data['xmin'] / width 
    data.loc[:,'xmax'] = data['xmax'] / width
    data.loc[:,'ymax'],data.loc[:,'ymin']  = (height - data['ymin']) / height , (height - data['ymax']) / height
    
    return data

In [ ]:
import pandas as pd
import numpy as np
import time

# import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import model_from_json
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

from tqdm.keras import TqdmCallback



# ----------- import data and scaling ----------- #
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

X_train = df_train[['ymin','xmin','ymax', 'xmax']]
y_train = df_train[['xloc','zloc']].values

X_test = df_test[['ymin','xmin','ymax', 'xmax']]
y_test = df_test[['xloc','zloc']].values


X_train = norm(X_train).values
X_test = norm(X_test).values

In [ ]:

print(X_train[:10])


In [ ]:
# TRAINING MODE


# ----------- create model ----------- #
model = keras.Sequential()
model.add(Dense(100, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(Dense(100, kernel_initializer='normal', activation='relu'))
model.add(Dense(100, kernel_initializer='normal', activation='relu'))
model.add(Dense(2, kernel_initializer='normal'))


# opt = tf.keras.optimizers.Adam(
#     learning_rate=0.001
# )
model.compile(loss='mse',optimizer='adam',metrics=['mae', 'mse'])

# ----------- define callbacks ----------- #
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7,
                verbose=1, min_delta=1e-4, mode='min')
modelname = "model@{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir="logs/{}".format(modelname))

# ----------- start training ----------- #
history = model.fit(X_train, y_train,
            validation_split=0.2, epochs=10000, batch_size=1024,verbose=0, callbacks=[tensorboard, TqdmCallback(verbose=1)])


# ----------- save model and weights ----------- #
model_json = model.to_json()
with open("generated_files/{}.json".format(modelname), "w") as json_file:
   json_file.write(model_json)

model.save_weights("generated_files/{}.h5".format(modelname))
print("Saved model to disk")



In [ ]:
pip install git+https://github.com/tensorflow/docs

In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)


In [ ]:
plotter.plot({'Basic': history}, metric = "loss")
plt.ylim([0, 10])
plt.ylabel('MAE [MPG]')

plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "val"], loc="upper left")
plt.show()

In [ ]:
# INFERENCE MODE

from tensorflow.keras.models import model_from_json

df_test = pd.read_csv('data/test.csv')
X_test = df_test[['ymin', 'xmin', 'ymax','xmax']]
y_test = df_test[['xloc' ,'zloc']].values

    
X_test = norm(X_test).values


# MODEL = "model@1597671595"
# WEIGHTS = "model@1597671595"

MODEL = modelname
WEIGHTS = modelname


# load json and create model
json_file = open('generated_files/{}.json'.format(MODEL), 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json( loaded_model_json )

# load weights into new model
loaded_model.load_weights("generated_files/{}.h5".format(WEIGHTS))
print("Loaded model from disk")

# loaded_model = model
# evaluate loaded model on test data
loaded_model.compile(loss='mean_squared_error', optimizer='adam')
y_pred = loaded_model.predict(X_test)


# save predictions
df_result = df_test
df_result['zloc_pred'] = -100000
df_result['xloc_pred'] = -100000


for idx, row in df_result.iterrows():
    df_result.at[idx, 'zloc_pred'] = y_pred[idx][2]
    df_result.at[idx, 'xloc_pred'] = y_pred[idx][0]

df_result.to_csv('data/predictions2.csv', index=False)

In [ ]:

categories = [{
        'id': 1,
        'name': 'car'
    }, {
        'id': 2,
        'name': 'van'
    }, {
        'id': 3,
        'name': 'truck'
    }, {
        'id': 4,
        'name': 'pedestrian'
    }]

cat_index  = {i+1: val for i,val in enumerate(categories) }

In [ ]:

def run_locnet( bboxes):
  
    if len(bboxes)==0 or len(bboxes)==1 and not any(bboxes[0]):
        return []
    y_pred = loaded_model.predict(bboxes)
  
    return np.hstack((bboxes,y_pred))


In [ ]:
def transform_to_kitti(classes):
    
    hash = { 3:1 , 8:3, 1:4, 7:7 }
    return [ hash.get(classes[i], 8) for i in range(len(classes)) ]

In [ ]:
def filter_detections( output_dict):
    
    output_dict['detection_classes'] = transform_to_kitti(output_dict['detection_classes'])
    
#     print(output_dict  )
    misc_ids = (8,9)
    
    scores = output_dict['detection_scores']
    classes = output_dict['detection_classes']
    
    size = len(classes)
    min_threshhold = 0.5
    output_dict['detection_boxes'] = np.array([ output_dict['detection_boxes'][i] for i in range(size) if scores[i] >= min_threshhold and classes[i] not in misc_ids  ])
    output_dict['detection_classes'] = np.array([ output_dict['detection_classes'][i] for i in range(size) if scores[i] >= min_threshhold and classes[i] not in misc_ids  ])
    output_dict['detection_scores'] = np.array([ output_dict['detection_scores'][i] for i in range(size) if scores[i] >= min_threshhold and classes[i] not in misc_ids  ])


    return output_dict

In [ ]:
def scale_up_boxes( output_dict):
    
    output_dict['detection_boxes'] = output_dict['detection_boxes'] * 1000

    return output_dict

In [ ]:
!pip install filterpy

In [ ]:
# initialize Sort object
from sort import *
mot_tracker = Sort( max_age=8, iou_threshold=0.3)


In [ ]:
def reorder_for_sort(array):

     return np.array( [array[1] ,array[0], array[3], array[2]])

In [ ]:
def format_detections_for_mot(outputdict):
    
    detections = [ np.append( reorder_for_sort(outputdict['detection_boxes'][i]) , outputdict['detection_scores'][i])  for i in range( len(outputdict['detection_classes'])) ] 
    
    return np.asarray(detections)  if len(detections) else np.empty((0, 5))


In [ ]:
def format_boxes( boxes):
    
    result = boxes.numpy()
    for box in result:
        box[0]=1-box[0]
        box[2]=1-box[2]
        box[0],box[2] = box[2], box[0]
    return tf.convert_to_tensor(result)

In [ ]:
def get_tracked_color(label):
    
    return vis_util.STANDARD_COLORS[label %len(vis_util.STANDARD_COLORS)]

In [306]:
import copy

def process_image(image, objects=None, groundtruth=None, mode=None):
  

    image_np = np.copy(image) #TODO: make more efficent 
    
    output_dict = run_inference_for_single_image(detection_model, image_np)
    
    output_dict =filter_detections( output_dict)

    mode ='tracking'
    if mode =='tracking':
        
        detections = format_detections_for_mot(output_dict)
        #     print( 'detections', detections)
        tracked_objects = mot_tracker.update(detections)
        for kt in mot_tracker.trackers:
            
            k = copy.deepcopy(kt)
            
            if len(k.history)>=3:
                for x in range(3):
                    k.predict()

                xmin,ymin,xmax,ymax = k.get_state()[0]
                track_id = k.id
                label = str(track_id)
                vis_util.draw_bounding_box_on_image_array(image_np, ymin, xmin, ymax, xmax, thickness=1, display_str_list=[label],color=get_tracked_color(track_id))

            print(  'id' ,kt.id, 'history' , kt.history, 'hits' , kt.hits, 'hit_streak' , kt.hit_streak , 'age' ,kt.age )
        
        for object in tracked_objects:
            xmin,ymin,xmax,ymax = object[:4]
            track_id = int(object[4])
#             n= track_id%40 +1
            n= track_id
            label = f'object{n:03}'
            vis_util.draw_bounding_box_on_image_array(image_np, ymin, xmin, ymax, xmax, thickness=1, display_str_list=[label],color=get_tracked_color(track_id))
            
       
    elif mode=='locations':
        locations = run_locnet( output_dict['detection_boxes'])

        i=0
        while i < len(objects['location']) and i < len(locations):
            ymin,xmin,ymax,xmax = locations[i][:4]
    #             print( objects['location'].numpy() , i)
            coords = locations[i][4:7:2]
            groundtruth_coords = objects['location'].numpy()[i][0:3:2]
            label = "predicted:(%.1f,%.1f), actual:(%.1f,%.1f)" % tuple(np.append(coords  , groundtruth_coords).tolist())
            vis_util.draw_bounding_box_on_image_array(image_np, ymin, xmin, ymax, xmax, thickness=1, display_str_list=[label],color='green')
            i+=1

            
    else:
        # Visualization of the results of a detection.

        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          cat_index,
          use_normalized_coordinates=True,
          line_thickness=2)


        if groundtruth:
            groundtruth_boxes = format_boxes(objects['bbox']).numpy()
            groundtruth_classes = objects['type'].numpy()+1
            vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          groundtruth_boxes,
          groundtruth_classes,
            None,
          cat_index,
          use_normalized_coordinates=True,
          line_thickness=1,
            groundtruth_box_visualization_color='blue')
        
    
    return image_np

In [ ]:
run_locnet( [[0.41085333,0.4335749 ,0.5223467 ,0.4712963 ]])

In [ ]:

# Load the TensorBoard notebook extension
# %reload_ext tensorboard


%tensorboard --logdir logs


In [ ]:
%reload_ext tensorboard

In [ ]:
!pip install tensorboard 

In [ ]:
def show_inference(model, tensor, objects, groundtruth=None, mode=None):
  
    image_np = np.array(tensor)
    image =process_image(image_np, objects, groundtruth, mode)
    display(Image.fromarray(image))
    return

Experiments

In [ ]:

for example in ds_test.take(50):  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
    image = example["image"]
    objects = example["objects"]
  
    print('bbox:' ,objects['bbox'])
    print('location:', objects['location'])
    print('type:', objects['type'])
    show_inference(detection_model, image, objects, mode='locations')
    

In [ ]:
print(len(ds_test))

In [ ]:
import time

total = 0
for example in ds_test:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
#     print(list(example.keys()))
    image = example["image"]
    objects = example["objects"]
  
#     print('bbox:' ,objects['bbox'])
#     print('location:', objects['location'])
#     print('type:', objects['type'])
#     show_inference(detection_model, image, objects, tracking_mode=False)
    
    start = time.time()
    image_np = np.array(image)
    run_inference_for_single_image(detection_model, image_np)
    end = time.time()
    total += (end - start)
    
print( total/len(ds_test))



## Notes on bbox from tfds

bbox: tf.Tensor of type `tf.float32` and shape `[4,]` which contains the
      normalized coordinates of the bounding box `[ymin, xmin, ymax, xmax]`

In [ ]:
cat_index

In [ ]:
#QUALITATIVE EVALUATION STEPS

from object_detection import eval_util
from object_detection.core import standard_fields as fields
from object_detection.metrics import coco_evaluation
from object_detection.protos import eval_pb2
from object_detection.utils import test_case
from object_detection.utils import tf_version

input_data_fields = fields.InputDataFields
detection_fields = fields.DetectionResultFields


for example in ds_test.take(15):  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
    image = example["image"]
    objects = example["objects"]
#     show_inference(detection_model, image)
    image_np = np.array(image)
    # Actual detection.
    output_dict = run_inference_for_single_image(detection_model, image_np)


    output_dict = filter_detections( output_dict)

    batch_size = 1
    key=tf.constant('image1')
    
    groundtruth_boxes = format_boxes(objects['bbox'])
    groundtruth_classes = objects['type']+1
    groundtruth = {
        input_data_fields.groundtruth_boxes: groundtruth_boxes,
        input_data_fields.groundtruth_classes: groundtruth_classes,      
    }
    
    num_detections = tf.constant([len(output_dict['detection_classes'])])
    
    detections = {
        detection_fields.detection_boxes: tf.constant([output_dict['detection_boxes']]) ,
        detection_fields.detection_scores: tf.constant([output_dict['detection_scores']]),
        detection_fields.detection_classes: tf.constant([output_dict['detection_classes']]),
        detection_fields.num_detections: num_detections
       
    }
    

    image = tf.constant([image_np])
    
    
    result_dict = eval_util.result_dict_for_single_example(image, key,detections, groundtruth)
    
    side_by_side_img =  vis_util.draw_side_by_side_evaluation_image(result_dict,cat_index)[0][0].numpy()
    display(Image.fromarray(side_by_side_img))


In [ ]:
category_index

In [ ]:
result_dict

In [ ]:
#TODO: Complete Evaluation steps
from object_detection.core import standard_fields
from object_detection.utils import object_detection_evaluation


categories = [{
        'id': 1,
        'name': 'car'
    }, {
        'id': 4,
        'name': 'pedestrian'
    }]

# print( categories)
pascal_evaluator = object_detection_evaluation.PascalDetectionEvaluator(
        categories,matching_iou_threshold=0.5)

# ,evaluate_precision_recall=True

for index, example in enumerate(ds_test):  
    image = example["image"]
    objects = example["objects"]
  
#     show_inference(detection_model, image)
    image_np = np.array(image)
    # Actual detection.
    output_dict = run_inference_for_single_image(detection_model, image_np)
    


    output_dict = filter_detections( output_dict)
    
#     output_dict = scale_up_boxes( output_dict)
    
    image_key = 'img'+str(index)
    

#     groundtruth_boxes = objects['bbox'].numpy()
    groundtruth_boxes = format_boxes(objects['bbox']).numpy()
    groundtruth_classes = objects['type'].numpy()+1
    
#     print( 'ground_truth_classes' ,  groundtruth_classes)
#     print( 'ground_truth_boxes' ,  groundtruth_boxes)
#     print( 'detection_classes' ,  output_dict['detection_classes'])
#     print( 'detection_boxes' ,  output_dict['detection_boxes'])
#     print( 'detection_scores' ,  output_dict['detection_scores'])
    
#     print( '*'*100)

    pascal_evaluator.add_single_ground_truth_image_info(
        image_key,
        {standard_fields.InputDataFields.groundtruth_boxes: groundtruth_boxes,
         standard_fields.InputDataFields.groundtruth_classes:
         groundtruth_classes,
#          standard_fields.InputDataFields.groundtruth_difficult:
#          groundtruth_is_difficult_list2
        })

    if len(output_dict['detection_classes']):
        pascal_evaluator.add_single_detected_image_info(
                image_key,
                {standard_fields.DetectionResultFields.detection_boxes: output_dict['detection_boxes'],
                 standard_fields.DetectionResultFields.detection_scores:
                 output_dict['detection_scores'],
                 standard_fields.DetectionResultFields.detection_classes:
                 output_dict['detection_classes']
                })
    

    metrics = pascal_evaluator.evaluate()
    
    


In [ ]:
output_dict

In [ ]:
metrics

In [ ]:
!pip install moviepy proglog

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# !gsutil cp gs://hazard-detection-test-videos/accident_compilation.mp4 accident_compilation.mp4

In [ ]:
!ls -sh accident_compilation.mp4

In [307]:
from moviepy.editor import VideoFileClip
import proglog
proglog.notebook()

mot_tracker = Sort( max_age=8, iou_threshold=0.5)

tracked_ids=None

write_output = 'output_' + model_name + str(time.time())+ '.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first n seconds
# clip1 = VideoFileClip("test.mp4").subclip(0,1)

# clip1 = VideoFileClip("test.mp4")

clip1 = VideoFileClip("accident_compilation.mp4").subclip(59,69)


white_clip = clip1.fl_image(process_image) 

white_clip.write_videofile(write_output, audio=False, verbose=False)

clip1.close()


id 4901 history [] hits 0 hit_streak 0 age 0
id 4902 history [] hits 0 hit_streak 0 age 0
id 4903 history [] hits 0 hit_streak 0 age 0
Moviepy - Building video output_faster_rcnn_resnet50_lowproposals_coco_2018_01_281597781882.6326866.mp4.
Moviepy - Writing video output_faster_rcnn_resnet50_lowproposals_coco_2018_01_281597781882.6326866.mp4



id 4901 history [] hits 1 hit_streak 1 age 1
id 4902 history [] hits 1 hit_streak 1 age 1
id 4903 history [] hits 1 hit_streak 1 age 1
id 4901 history [] hits 2 hit_streak 2 age 2
id 4902 history [] hits 2 hit_streak 2 age 2
id 4903 history [] hits 2 hit_streak 2 age 2
id 4904 history [] hits 0 hit_streak 0 age 0
id 4901 history [] hits 3 hit_streak 3 age 3
id 4902 history [] hits 3 hit_streak 3 age 3
id 4903 history [] hits 3 hit_streak 3 age 3
id 4904 history [array([[0.0326736 , 0.41040734, 0.84744906, 1.00000003]])] hits 0 hit_streak 0 age 1
id 4901 history [] hits 4 hit_streak 4 age 4
id 4902 history [] hits 4 hit_streak 4 age 4
id 4903 history [] hits 4 hit_streak 4 age 4
id 4904 history [array([[0.0326736 , 0.41040734, 0.84744906, 1.00000003]]), array([[0.0326736 , 0.41040734, 0.84744906, 1.00000003]])] hits 0 hit_streak 0 age 2
id 4901 history [] hits 5 hit_streak 5 age 5
id 4902 history [] hits 5 hit_streak 5 age 5
id 4903 history [] hits 5 hit_streak 5 age 5
id 4904 history [

In [ ]:
# !gsutil cp gs://hazard-detection-test-videos/tf2model/* ./

In [ ]:
!ls checkpoint_dir

## LocNet starts
